# Cosmos Shell

The Cosmos Shell, is an acronym + visual pun.

    Cosmos: "Cascading Operating SysteM Operating System"
    Shell: "@ + specialized shell behavior"
    
## Wraps APath

The Cosmos Shell python object `CosmosShell` wraps an `APath` Access Path object. The shell uses the Access Path as a context to execute commands.

The Cosmos type-system has commands like `new` which create a file system object from a type template.


## Target Paths vs. APath Location

In the examples below, a target path may be specified like `/a/b/c.typename`. A CosmosShell operates only on one segment of a path. The path lookup mechanism can use CosmosShell to execute commands in each lookup step, in the context of each path segment, the resulting `APath` object for `/a/b/` would issue a command `new` to create `c.typename`. This command would run in a CosmosShell wrapping the `/a/b/` APath. The pathname value of the APath would be `/a/b/` and the new target directory entry pathname would be `/a/b/c.typename` so the target pathname could be pattern matched with a type-match of `*.typename`. However, it must be noted that the actual execution environment of the CosmosShell is the APath for `/a/b/`, and not `/`.


## Snail Shell Directories

    @/

Anywhere in the file system where there is an '@' directory, the contents of the '@' directory overload the contents of a previous '@' directory further up the hierarchy.

In an '@' aware file system, '@' will always resolve as a local directory, even if it doesn't exist in the underlying file system.

    .@/
    
The snail shell directory may be hidden, but functions the same.


## Object Oriented Type System

    @/Types/

A type system that uses the python `Jinja` template processor on filenames and file contents:
    
`@/Types` defines classes that may be instantiated under the APath's level of the file system hierarchy.

Example:

    /my/projects/@/Types/project
    
Enables...

    > new /my/project/special.project
    
...to instantiate a directory called `special.project/` which is prepopulated with files from `@/Types/project`

The `new` command accepts arguments that will expand to variable names within the `Types/project` template processed by `Jinja`.

    @/Types/project/
        {{main_file}}.py
        
Where `{{main_flename}}.py` has these contents:

    import {{class}} from {{module}}
    
    x = {{class}}({{ttl}})
    
And where `new` was invoked like this:

    > new /my/project/special.project --json '{"main_file":"boris","class":"Bomb","module":"coat_pocket","ttl":"5000"}'
    
Creates:

    /my/project/
        special.project/
            boris.py
            
Where `boris.py` has these contents:

    import Bomb from coat_pocket
    
    x = Bomb(5000)
    

## Python Implementation

When the `new` command is implemented in Python:

    if __name__ == "__main__":
        fs = OSFS('.')
        snail_shell = SnailShellFS(fs)
        


In [21]:
class CosmosShell:
    def __init__(self, apath=None):
        self._apath = apath
        return None
    
    def new(self, dirent_name='', typename='', **kwargs):
        return None

In [22]:
# Test CosmosShell Init

cosh = CosmosShell()

In [23]:
cosh

In [19]:
cosh.new('rclone.conf','*/.config/rclone/rclone.conf')

In [24]:
cosh.new('dbox','*/.config/rclone/rclone.conf/*.dropbox')

In [ ]:
# Test Apath Init Helper for CLI Invocation

CosmosShell.APath_from_cli()